In [1]:
import pandas as pd
import sqlite3
import os
import numpy as np
from datetime import datetime, timedelta,date


In [2]:
# data_base_file=r'E:\AB_DB\Script_ImportData\Database\DB_BN_MyTrade.db' # production
data_base_file=r'D:\AB_DB\Script_ImportData\Database\DB_BN_MyTrade.db' # dev
print("Connect SQLite")
conn = sqlite3.connect(os.path.abspath(data_base_file))

#sql=f"select * from hist_trade   order by datetimeUTC"
sql=f"select * from hist_trade  order by datetimeUTC"

print(sql)
df_trasns=pd.read_sql_query(sql, conn)
df_trasns['datetimeUTC']=pd.to_datetime(df_trasns["datetimeUTC"],format="%Y-%m-%d %H:%M:%S")
print(df_trasns.info())
df_trasns.tail()

Connect SQLite
select * from hist_trade  order by datetimeUTC
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   datetimeUTC      959 non-null    datetime64[ns]
 1   symbol_pair      959 non-null    object        
 2   symbol           959 non-null    object        
 3   pair             959 non-null    object        
 4   type             959 non-null    object        
 5   price            959 non-null    float64       
 6   qty              959 non-null    float64       
 7   total            959 non-null    float64       
 8   commission       959 non-null    float64       
 9   commissionAsset  959 non-null    object        
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 75.0+ KB
None


,datetimeUTC,symbol_pair,symbol,pair,type,price,qty,total,commission,commissionAsset
954,2022-10-13 16:45:14,MATICBUSD,MATIC,BUSD,SELL,0.770,20.0000,15.400000,0.0,BNB
955,2022-10-13 18:02:03,ETHBUSD,ETH,BUSD,BUY,1275.270,0.0117,14.920659,0.0,BNB
956,2022-10-14 16:09:44,BTCBUSD,BTC,BUSD,BUY,19300.000,0.0011,21.230000,0.0,BNB
957,2022-10-14 16:09:46,BNBBUSD,BNB,BUSD,BUY,270.200,0.0650,17.563000,0.0,BNB
958,2022-10-14 21:17:20,MATICBUSD,MATIC,BUSD,BUY,0.784,20.0000,15.680000,0.0,BNB


In [3]:
from google.cloud import bigquery
table_id = "pongthorn.FinDW.dev_bn_hist_trade"
def insertTradeToBQ(df_trasns):
    client = bigquery.Client()
    job_config = bigquery.LoadJobConfig(
#         schema=[
#             bigquery.SchemaField("datetimeUTC", bigquery.enums.SqlTypeNames.DATETIME),
#             bigquery.SchemaField("symbol_pair", bigquery.enums.SqlTypeNames.STRING),
#             bigquery.SchemaField("symbol", bigquery.enums.SqlTypeNames.STRING),
#             bigquery.SchemaField("pair", bigquery.enums.SqlTypeNames.STRING),
#             bigquery.SchemaField("type", bigquery.enums.SqlTypeNames.STRING),
#             bigquery.SchemaField("price", bigquery.enums.SqlTypeNames.FLOAT64),
#             bigquery.SchemaField("qty", bigquery.enums.SqlTypeNames.FLOAT64),
#             bigquery.SchemaField("total", bigquery.enums.SqlTypeNames.FLOAT64),
#             bigquery.SchemaField("commission", bigquery.enums.SqlTypeNames.FLOAT64),
#             bigquery.SchemaField("commissionAsset", bigquery.enums.SqlTypeNames.STRING),

#         ],
        # write_disposition="WRITE_APPEND",
        write_disposition="WRITE_TRUNCATE",
    )

    job = client.load_table_from_dataframe(
        df_trasns, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

In [4]:
insertTradeToBQ(df_trasns)